In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from skimage import io, transform

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%matplotlib inline

import timeit, random
import json, cv2
import show

%load_ext autoreload
%autoreload 2

# Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
def transform_joint(joints):
    j_2 = []
    for J in joints:
        j_2.append([int(J[0]//2), int(J[1]//2)])
    return j_2

def center(joints):
    min_x = min(joints, key=lambda x : x[0])[0]
    max_x = max(joints, key=lambda x : x[0])[0]
    min_y = min(joints, key=lambda x : x[1])[1]
    max_y = max(joints, key=lambda x : x[1])[1]
    return (int((max_x-min_x)//2 + min_x), int((max_y-min_y)//2 + min_y))

In [23]:
joints = json.load(open("Dataset/annotation.json","r"))
names = joints.keys()

In [5]:
names = list(names)
print(type(names), names[:10])

<class 'list'> ['007_1499_L', '072_2297_L', '066_1175_L', '028_389_R', '026_2063_R', '038_1673_R', '136_2045_L', '041_653_L', '026_2063_L', '066_1175_R']


In [ ]:
unsaved = []
for i, name in enumerate(names):
    image = cv2.imread("Dataset/Color/" + name[:-2] + ".jpg")
    small = cv2.resize(image, (0,0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA) # resize by half
    j_2 = transform_joint(joints[name])
    _x, _y = center(j_2)
    cropped = small[_y-64:_y+64 , _x-64:_x+64]
    
    image = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)

    u,v,d = image.shape
    
    if u != 128:
        unsaved.append(name)
    elif v != 128:
        unsaved.append(name)

    cv2.imwrite('Dataset/ColoredProcessedImages/' + name + '.png', image)
        
    if i%10000 == 0:
        print(i)
#     fig, ax = plt.subplots(1,1, figsize=(10, 10))
#     ax.imshow(image)
#     plt.show()
# average pooling vs max pooling.


0
10000
20000
40000


In [3]:
scaled_joints = json.load(open("Dataset/scaled_annotations.json","r"))
names = list(scaled_joints.keys())

In [36]:
resize = []
for i, name in enumerate(names):
    image = cv2.imread("Dataset/ColoredProcessedImages/" + name + ".png")
    if image.size != 49152:
        print(image.size)
        resize.append(name)

In [33]:
len(resize)

461

In [34]:
resize[0:10]

['065_401_R',
 '090_71_R',
 '090_71_L',
 '062_41_R',
 '062_29_R',
 '099_59_R',
 '099_197_R',
 '059_137_R',
 '003_2687_R',
 '003_2687_L']

In [35]:
for i, name in enumerate(resize):
    image = cv2.imread("Dataset/Color/" + name[:-2] + ".jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    small = cv2.resize(image, (0,0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA) # resize by half
    j_2 = transform_joint(joints[name])
    _x, _y = center(j_2)
    cropped = small[_y-64:_y+64 , _x-64:_x+64]
    

    
    u,v,d = cropped.shape
#     print(cropped.shape)

    
    if u != 128:
        cropped2 = small[small.shape[0]-128:small.shape[0] , _x-64:_x+64]
        cv2.imwrite('Dataset/ColoredProcessedImages/' + name + '.png', cropped2)

#     fig, ax = plt.subplots(1,2, figsize=(10, 10))
#     ax[0].imshow(cropped)
#     ax[1].imshow(cropped2)
#     plt.show()